This chapter covers
- The Keras functional API
- Using Keras callbacks
- Working with the TensorBoard visualization tool
- Important best practices for developing SOTA models

本章是关于Keras高级特性的介绍，使得你在应对deep-learning practice时更加得心应手。实践中包括了batch normalization, residual connections, hyperparameter optimization and model ensembling.

### 7.1 Going beyond the Sequential model: the Keras functional API

到现在为止，所有的模型都建立在Sequential模型之上：其假设网络只有一个输入和一个输出，且都是layer的线性堆叠。

虽然大多数网络的配置都是Sequential类型，但在某些情况下这一假设过于死板。有的网络需要多个独立的输入，有的网络则需要多个输出，而有些网络在layer和layer之间有内部的分支，这**使得网络的拓扑结构更像是图**，而非线性堆叠。

例如，some tasks require multimodal inputs:任务合并来自不同输入源的数据，并用不同类型的神经层处理不同类型的数据。可能同时利用metadata的features、图像信息的商品照片和文本信息的商品描述去预测一件二手衣服的价格。如果我们用naive方法训练三个独立模型，然后对三者的预测做加权平均，那么模型提供的信息可能是redundant的。更好的方法是，使用一个可以同时查看所有可用的输入modal的模型，从而联合学习一个(jointly learn)更加精确的数据模型——其有着三个input branches。也有着需要多个输出(multiple heads)的应用需求。

此外，许多最新开发的神经网络架构要求非线性的网络拓扑结构，即网络结构为有向无环图。比如Inception网络中的Inception modules和ResNet系列网络中的residual connection等。

这些重要的use cases——multi-input models, multi-output models, graph-like models是无法用sequential模型coding的。但Keras提供了一种more general, flexible的方法：functional API

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))

dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

C:\Users\10750\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


Model对象实例化只用了一个input_tensor和一个output_tensor。这是因为Keras会在后台检索从input_tensor到output_tensor 所包含的每一层，并将这些层组合成一个类图的数据结构，即一个Model。当然，这种方法有效的原因在于，output_tensor 是通过对input_tensor 进行多次变换得到的。如果你试图利用不相关的输入和输出来构建一个模型，那么会得到RuntimeError。

In [3]:
unrelated_input = Input(shape=(32, ))
bad_model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(?, 64), dtype=float32) at layer "input_2". The following previous layers were accessed without issue: []

In [4]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

socre = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 20s 20ms/step - loss: 11.8543
Epoch 2/10
1000/1000 [==============================] - 0s 54us/step - loss: 11.5996
Epoch 3/10
1000/1000 [==============================] - 0s 86us/step - loss: 11.5695
Epoch 4/10
1000/1000 [==============================] - 0s 61us/step - loss: 11.5608
Epoch 5/10
1000/1000 [==============================] - 0s 60us/step - loss: 11.5546
Epoch 6/10
1000/1000 [==============================] - 0s 53us/step - loss: 11.5509
Epoch 7/10
1000/1000 [==============================] - 0s 66us/step - loss: 11.5453
Epoch 8/10
1000/1000 [==============================] - 0s 92us/step - loss: 11.5420
Epoch 9/10
1000/1000 [==============================] - 0s 81us/step - loss: 11.5385
Epoch 10/10
1000/1000 [==============================] - 0s 105us/step


#### 7.1.2 Multi-Input models

